### popgen

Functions for creating founder populations in linkage equilbrium/disequilbrium

In [1]:
#| default_exp popgen

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| echo: false
from nbdev.showdoc import show_doc

In [4]:
#| export
from dataclasses import field
from typing import List, Optional, Dict, Callable, Any

import jax
import jax.numpy as jnp

from chewc.sp import SimParam
from typing import Tuple
from numpy.random import default_rng
import msprime
import tskit
import numpy as np
import random
from collections import defaultdict
from flax.struct import dataclass as flax_dataclass, field

#testing
import jax
import jax.numpy as jnp
from fastcore.test import test_eq, test_ne

In [5]:
#| export
from typing import Tuple
import jax
import jax.numpy as jnp
from chewc.population import Population


def quick_haplo(
    key: jax.random.PRNGKey,
    n_ind: int,
    n_chr: int,
    n_loci_per_chr: int,
    max_pop_size: int,
    ploidy: int = 2,
    inbred: bool = False,
    chr_len_cm: float = 100.0
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a new, padded founder population with random haplotypes.

    Args:
        key: A JAX random key.
        n_ind: Number of founder individuals to create.
        n_chr: Number of chromosomes.
        n_loci_per_chr: Number of loci on each chromosome.
        max_pop_size: The total size of the arrays to pre-allocate.
        ... (other args are the same)

    Returns:
        A tuple of (Population, genetic_map). The Population object will
        contain `n_ind` active individuals in arrays of size `max_pop_size`.
    """
    if n_ind > max_pop_size:
        raise ValueError("n_ind cannot be greater than max_pop_size.")

    key, geno_key, sex_key = jax.random.split(key, 3)

    # 1. Generate data for the initial n_ind founders
    if inbred:
        base_haplotypes = jax.random.randint(geno_key, (n_ind, n_chr, 1, n_loci_per_chr), 0, 2, dtype=jnp.uint8)
        founder_geno = jnp.tile(base_haplotypes, (1, 1, ploidy, 1))
    else:
        founder_geno = jax.random.randint(geno_key, (n_ind, n_chr, ploidy, n_loci_per_chr), 0, 2, dtype=jnp.uint8)
    
    founder_ids = jnp.arange(n_ind)
    founder_sex = jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,))
    
    n_founder_alleles = n_ind * n_chr * ploidy * n_loci_per_chr
    founder_ibd_flat = jnp.arange(n_founder_alleles, dtype=jnp.uint32)
    founder_ibd = founder_ibd_flat.reshape(n_ind, n_chr, ploidy, n_loci_per_chr)
    if inbred:
        founder_ibd = jnp.tile(founder_ibd[:, :, 0:1, :], (1, 1, ploidy, 1))

    # 2. Pad all arrays to max_pop_size
    n_pad = max_pop_size - n_ind
    
    padded_geno = jnp.pad(founder_geno, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=0)
    padded_ibd = jnp.pad(founder_ibd, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=-1)
    padded_id = jnp.pad(founder_ids, (0, n_pad), constant_values=-1)
    padded_sex = jnp.pad(founder_sex, (0, n_pad), constant_values=-1)

    # 3. Create the is_active mask
    is_active = jnp.arange(max_pop_size) < n_ind

    # --- Generate a uniform genetic map ---
    loci_pos = jnp.linspace(0., chr_len_cm, n_loci_per_chr)
    genetic_map = jnp.tile(loci_pos, (n_chr, 1))

    population = Population(
        geno=padded_geno,
        ibd=padded_ibd,
        id=padded_id,
        iid=jnp.arange(max_pop_size), # iid is always contiguous for indexing
        mother=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        father=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        sex=padded_sex,
        gen=jnp.zeros(max_pop_size, dtype=jnp.int32), # Active individuals are gen 0
        is_active=is_active,
        pheno=jnp.full((max_pop_size, 0), jnp.nan),
        fixEff=jnp.zeros(max_pop_size, dtype=jnp.float32),
        bv=jnp.full((max_pop_size, 0), jnp.nan),
    )
    
    return population, genetic_map


In [6]:
#| export

def msprime_pop(
    key: jax.random.PRNGKey,
    n_ind: int,
    n_loci_per_chr: int,
    n_chr: int,
    max_pop_size: int,
    ploidy: int = 2,
    effective_population_size: int = 10_000,
    mutation_rate: float = 2e-8,
    recombination_rate_per_chr: float = 2e-8,
    maf_threshold: float = 0.1,
    num_simulated_individuals: int = None,
    base_chr_length: int = 1_000_000,
    enforce_founder_maf: bool = True
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a new, padded founder population using msprime coalescent simulation.

    Args:
        key: JAX random key.
        n_ind: Number of founder individuals to generate.
        n_loci_per_chr: Number of SNPs (loci) to select per chromosome.
        n_chr: Number of chromosomes.
        max_pop_size: The total size of the arrays to pre-allocate for JAX compatibility.
        ... (other args remain the same) ...

    Returns:
        A tuple containing:
        - A new Population object with `n_ind` active founders in padded arrays of size `max_pop_size`.
        - A JAX array representing the genetic map.
    """
    # --- Parameter Validation ---
    if n_ind > max_pop_size:
        raise ValueError(f"Number of founders ({n_ind}) cannot exceed max_pop_size ({max_pop_size}).")
        
    if effective_population_size > 100_000:
        raise ValueError(f"Effective population size {effective_population_size} is very large and may cause memory issues.")
    
    if num_simulated_individuals is None:
        multiplier = 5 if enforce_founder_maf else 2
        num_simulated_individuals = min(max(n_ind * multiplier, 1000), 10_000)
        
    if n_ind > num_simulated_individuals:
        raise ValueError(f"Number of founders requested ({n_ind}) cannot exceed the base simulated population size ({num_simulated_individuals}).")

    # --- Simulation Setup (Unchanged) ---
    key, seed_key, sex_key, numpy_seed_key = jax.random.split(key, 4)
    random_seed = int(jnp.sum(seed_key))
    numpy_seed = int(jnp.sum(numpy_seed_key))
    rng = default_rng(numpy_seed)

    chromosome_lengths = [base_chr_length] * n_chr
    num_haplotypes = num_simulated_individuals * ploidy
    rate_map_positions = [0] + list(np.cumsum(chromosome_lengths))
    rate_map_rates = [recombination_rate_per_chr] * len(chromosome_lengths)
    rate_map = msprime.RateMap(position=rate_map_positions, rate=rate_map_rates)

    ts = msprime.sim_ancestry(
        samples=num_haplotypes,
        population_size=effective_population_size,
        recombination_rate=rate_map,
        random_seed=random_seed
    )
    mts = msprime.sim_mutations(ts, rate=mutation_rate, random_seed=random_seed)

    # --- Data Extraction (Unchanged) ---
    true_num_individuals = mts.num_samples // ploidy
    founder_indices = np.sort(rng.choice(true_num_individuals, n_ind, replace=False))
    
    all_variants = list(mts.variants())
    genetic_map = np.full((n_chr, n_loci_per_chr), np.nan)
    founder_haplotype_matrix = np.full((n_ind, n_chr, ploidy, n_loci_per_chr), np.nan)

    for i in range(n_chr):
        chr_start, chr_end = rate_map.left[i], rate_map.right[i]
        chromosome_snps = [v for v in all_variants if chr_start <= v.site.position < chr_end and len(v.alleles) == 2]

        if enforce_founder_maf:
            eligible_snps = [
                var for var in chromosome_snps 
                if min(np.mean(var.genotypes.reshape(true_num_individuals, ploidy)[founder_indices]), 
                       1 - np.mean(var.genotypes.reshape(true_num_individuals, ploidy)[founder_indices])) > maf_threshold
            ]
        else:
            eligible_snps = [var for var in chromosome_snps if min(np.mean(var.genotypes), 1 - np.mean(var.genotypes)) > maf_threshold]

        num_found = len(eligible_snps)
        num_to_select = min(num_found, n_loci_per_chr)

        if num_to_select > 0:
            selected_indices = rng.choice(len(eligible_snps), num_to_select, replace=False)
            selected_snps = [eligible_snps[i] for i in selected_indices]
            selected_snps.sort(key=lambda v: v.site.position)

            for snp_idx, snp in enumerate(selected_snps):
                all_genotypes = snp.genotypes.reshape(true_num_individuals, ploidy)
                founder_genotypes = all_genotypes[founder_indices]
                founder_haplotype_matrix[:, i, :, snp_idx] = founder_genotypes
            
            positions_cm = [(v.site.position - chr_start) * recombination_rate_per_chr * 100 for v in selected_snps]
            genetic_map[i, :num_to_select] = positions_cm

    # --- Padding and Final Population Assembly ---
    n_pad = max_pop_size - n_ind

    # Convert to JAX arrays before padding
    founder_geno = jnp.array(founder_haplotype_matrix, dtype=jnp.uint8)
    gen_map_jax = jnp.array(genetic_map)

    # Pad core genotype/IBD arrays
    padded_geno = jnp.pad(founder_geno, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=0)
    
    # Create and pad IBD data
    n_founder_alleles = n_ind * n_chr * ploidy * n_loci_per_chr
    founder_ids_ibd = jnp.arange(n_founder_alleles, dtype=jnp.uint32)
    founder_ibd = founder_ids_ibd.reshape(n_ind, n_chr, ploidy, n_loci_per_chr)
    padded_ibd = jnp.pad(founder_ibd, ((0, n_pad), (0, 0), (0, 0), (0, 0)), constant_values=-1)

    # Pad pedigree and identifier arrays
    padded_id = jnp.pad(jnp.arange(n_ind), (0, n_pad), constant_values=-1)
    padded_sex = jnp.pad(jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,)), (0, n_pad), constant_values=-1)
    
    # Create the is_active mask
    is_active = jnp.arange(max_pop_size) < n_ind

    pop = Population(
        geno=padded_geno,
        ibd=padded_ibd,
        id=padded_id,
        iid=jnp.arange(max_pop_size, dtype=jnp.int32),
        mother=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        father=jnp.full(max_pop_size, -1, dtype=jnp.int32),
        sex=padded_sex,
        gen=jnp.zeros(max_pop_size, dtype=jnp.int32),
        is_active=is_active,
        pheno=jnp.full((max_pop_size, 0), jnp.nan),
        fixEff=jnp.zeros(max_pop_size, dtype=jnp.float32),
        bv=jnp.full((max_pop_size, 0), jnp.nan),
    )

    return pop, gen_map_jax


In [8]:
#| hide
import nbdev; nbdev.nbdev_export()